<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Hisseler_XU100_Getiri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 Google Colab için Gerekli Kütüphaneleri Yükleme
!pip install tradingview_screener
!pip install git+https://github.com/rongardF/tvdatafeed matplotlib openpyxl
!pip install tradingview-screener==2.5.0
!pip install tqdm
!pip install ta

# 📌 Gerekli Kütüphaneler
try:
    import tradingview_screener
    from tradingview_screener import get_all_symbols
    import pandas as pd
    import numpy as np
    from tvDatafeed import TvDatafeed, Interval
    from tqdm import tqdm
    from datetime import datetime
    import warnings
    import ta
    from tabulate import tabulate
    # 📌 Uyarıları Engelle
    warnings.filterwarnings("ignore", category=FutureWarning)

except ImportError:
    print("Eksik kütüphaneler var, lütfen kurulumları tamamlayın.")
    exit()

# 📌 TradingView Bağlantısı
tv = TvDatafeed()

# 📌 Kullanıcıdan Girişlerin Alınması
print("📌 Hangi yıldan itibaren karşılaştırılsın? (Örnek: 2020, 2015, 2022)")
baslangic_yili = input("Lütfen bir yıl girin: ")

print("📌 Günlük mü, haftalık mı tarama yapılsın? (Günlük için 'G', Haftalık için 'H' girin)")
tarama_periyodu = input("Lütfen bir seçenek girin (G/H): ").strip().upper()

# 📌 Tarih Aralığı ve Interval Ayarları
baslangic_tarihi = f"{baslangic_yili}-01-01"
bitis_tarihi = datetime.now().strftime("%Y-%m-%d")
interval = Interval.in_daily if tarama_periyodu == "G" else Interval.in_weekly

# 📌 Minimum Veri Şartı (Örneğin, en az 1 yıllık veri)
MINIMUM_VERI_GUNU = 252 if tarama_periyodu == "G" else 52  # 1 yıl ≈ 252 iş günü veya 52 hafta

# 📌 BIST Hisselerini Al (Önceki Projeye Göre)
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# 📌 XU100 Verilerini Al
xu100_df = tv.get_hist("XU100", exchange="BIST", interval=interval, n_bars=1000)
xu100_df = xu100_df[(xu100_df.index >= baslangic_tarihi) & (xu100_df.index <= bitis_tarihi)]
xu100_getiri = (xu100_df["close"].iloc[-1] / xu100_df["close"].iloc[0] - 1) * 100

# 📌 Tarama Sonuçları DataFrame
tarama_sonuclari = pd.DataFrame(columns=['Hisse', 'Getiri (%)', 'XU100 Getiri (%)', 'Fark (%)'])
yetersiz_veri_hisseler = pd.DataFrame(columns=['Hisse', 'Getiri (%)', 'XU100 Getiri (%)', 'Fark (%)'])

# 📌 Tarama İşlemi
for hisse in tqdm(Hisseler, desc="📊 Hisseler Taranıyor", unit="hisse"):
    try:
        df = tv.get_hist(hisse, exchange='BIST', interval=interval, n_bars=1000)
        df = df[(df.index >= baslangic_tarihi) & (df.index <= bitis_tarihi)]

        # 📌 Minimum Veri Şartı Kontrolü
        if df is None or len(df) < MINIMUM_VERI_GUNU:
            hisse_getiri = (df["close"].iloc[-1] / df["close"].iloc[0] - 1) * 100
            fark = hisse_getiri - xu100_getiri
            yetersiz_veri_hisseler.loc[len(yetersiz_veri_hisseler)] = [
                hisse, round(hisse_getiri, 2), round(xu100_getiri, 2), round(fark, 2)
            ]
            continue

        # 📌 Getiri Hesaplama
        hisse_getiri = (df["close"].iloc[-1] / df["close"].iloc[0] - 1) * 100
        fark = hisse_getiri - xu100_getiri

        # 📌 Sonuçları DataFrame'e Ekle
        tarama_sonuclari.loc[len(tarama_sonuclari)] = [
            hisse, round(hisse_getiri, 2), round(xu100_getiri, 2), round(fark, 2)
        ]

    except Exception as e:
        print(f"{hisse} için hata: {e}")

# 📌 Tarama Sonuçlarını Analiz Et
yukselen_hisseler = tarama_sonuclari[tarama_sonuclari['Fark (%)'] > 0]
dusen_hisseler = tarama_sonuclari[tarama_sonuclari['Fark (%)'] <= 0]

# 📌 Bilgilendirme Mesajı
bilgilendirme_mesaji = (
    f"{baslangic_yili} yılından itibaren günümüze kadar geçen süre dikkate alındığında, "
    f"tarama sonucunda BİST Hisselerinden **{len(yukselen_hisseler)}** adeti XU100 Endeksi üzerinde getiri elde etmiştir. "
    f"**{len(dusen_hisseler)}** adet ise XU100 Endeksi altında kalarak getiri elde edememiştir."
)
print("\nℹ️ **Bilgilendirme Mesajı**")
print(bilgilendirme_mesaji)

# 📌 Kullanıcıdan Getiri Yüzdesi Değeri Al
getiri_yuzdesi = float(input("📈 **Getiri Yüzdesi, % Kaçın üzerinde olsun? (Örnek: 200):** "))

# 📌 Normal Tarama (Mevcut Tarama İşlemi)
print("\n📊 **Normal Tarama Sonuçları (XU100'e Göre)**")
print(tarama_sonuclari.sort_values(by="Fark (%)", ascending=False).to_string(index=False))

# 📌 Getiri Yüzdesine Göre Filtreleme
filtrelenmis_hisseler = tarama_sonuclari[tarama_sonuclari["Fark (%)"] >= getiri_yuzdesi]
print(f"\n📈 **{getiri_yuzdesi}% ve Üzeri Getiri Elde Eden Hisseler**")
print(filtrelenmis_hisseler.sort_values(by="Fark (%)", ascending=False).to_string(index=False))

# 📌 Artı Getiri Elde Edenler
artı_getiri_hisseler = tarama_sonuclari[tarama_sonuclari["Fark (%)"] > 0]
print("\n📈 **Artı Getiri Elde Eden Hisseler**")
print(artı_getiri_hisseler.sort_values(by="Fark (%)", ascending=False).to_string(index=False))

# 📌 Eksi Getiri Elde Edenler
eksi_getiri_hisseler = tarama_sonuclari[tarama_sonuclari["Fark (%)"] < 0]
print("\n📉 **Eksi Getiri Elde Eden Hisseler**")
print(eksi_getiri_hisseler.sort_values(by="Fark (%)", ascending=True).to_string(index=False))

# 📌 Yetersiz Veri Hisselerini Listele
print("\n📉 **Yetersiz Veri Hisseleri (1 Yıldan Az Verisi Olanlar)**")
print(yetersiz_veri_hisseler.sort_values(by="Fark (%)", ascending=False).to_string(index=False))

# 📌 En İyi ve En Kötü 30 Hisseyi Listele
en_iyi_30 = tarama_sonuclari.nlargest(30, 'Fark (%)')
en_kotu_30 = tarama_sonuclari.nsmallest(30, 'Fark (%)')

print("\n🏆 **En İyi 30 Hisse**")
print(en_iyi_30.to_string(index=False))

print("\n📉 **En Kötü 30 Hisse**")
print(en_kotu_30.to_string(index=False))

# 📌 Sonuçları Excel'e Kaydet
output_file = f"BIST_Getiri_Analizi_{baslangic_yili}_ile_{bitis_tarihi.replace('-', '')}.xlsx"
with pd.ExcelWriter(output_file) as writer:
    tarama_sonuclari.to_excel(writer, sheet_name="Ana_Tarama", index=False)
    filtrelenmis_hisseler.to_excel(writer, sheet_name=f"{getiri_yuzdesi}%_Uzeri", index=False)
    artı_getiri_hisseler.to_excel(writer, sheet_name="Artı_Getiri", index=False)
    eksi_getiri_hisseler.to_excel(writer, sheet_name="Eksi_Getiri", index=False)
    en_iyi_20.to_excel(writer, sheet_name="En_Iyi_30", index=False)
    en_kotu_20.to_excel(writer, sheet_name="En_Kotu_30", index=False)
    yetersiz_veri_hisseler.to_excel(writer, sheet_name="Yetersiz_Veri", index=False)

print(f"\n💾 Sonuçlar '{output_file}' dosyasına kaydedildi. ✅")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-3b_s5aei
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-3b_s5aei
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


📌 Hangi yıldan itibaren karşılaştırılsın? (Örnek: 2020, 2015, 2022)
Lütfen bir yıl girin: 2024
📌 Günlük mü, haftalık mı tarama yapılsın? (Günlük için 'G', Haftalık için 'H' girin)
Lütfen bir seçenek girin (G/H): h


📊 Hisseler Taranıyor: 100%|██████████| 601/601 [05:53<00:00,  1.70hisse/s]



ℹ️ **Bilgilendirme Mesajı**
2024 yılından itibaren günümüze kadar geçen süre dikkate alındığında, tarama sonucunda BİST Hisselerinden **225** adeti XU100 Endeksi üzerinde getiri elde etmiştir. **337** adet ise XU100 Endeksi altında kalarak getiri elde edememiştir.
📈 **Getiri Yüzdesi Kaç Olsun? (Örnek: 10.5):** 200

📊 **Normal Tarama Sonuçları (XU100'e Göre)**
Hisse  Getiri (%)  XU100 Getiri (%)  Fark (%)
TUREX      833.95             22.95    811.00
GRTHO      648.07             22.95    625.12
 UFUK      588.64             22.95    565.68
EUHOL      588.24             22.95    565.28
DERHL      535.23             22.95    512.27
PEHOL      528.00             22.95    505.04
RALYH      507.02             22.95    484.06
DAGHL      474.73             22.95    451.77
SANFM      450.24             22.95    427.29
IZFAS      414.19             22.95    391.23
SNKRN      401.13             22.95    378.18
BALAT      341.18             22.95    318.22
GZNMI      337.71             22.95    